In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
df = pd.read_parquet('../data/test_sac_model.parquet')
df = df.dropna(subset=['message']).reset_index(drop=True)

In [ ]:
df.to_json('../data/payload.json', orient='records')

In [ ]:
import boto3
from datetime import datetime

client = boto3.client(service_name='sagemaker')

model_name = 'sagemaker-scikit-learn-2022-01-17-17-31-53-523'
now = datetime.now()
current_time = now.strftime("%Y-%m-%d-%H-%M-%S")

create_batch = client.create_transform_job(
    TransformJobName='sac-emails-{}'.format(current_time),
    ModelName=model_name,
    # MaxConcurrentTransforms=0,
    MaxPayloadInMB=20,
    # BatchStrategy='MultiRecord',
    TransformInput={
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': 's3://sagemaker-us-east-1-296025910508/sac-emails/input/payload.json'
            }
        },
        'ContentType': 'application/json'
        # 'SplitType': 'Line'
    },
    TransformOutput={
        'S3OutputPath': 's3://sagemaker-us-east-1-296025910508/sac-emails/output/'
    },
    TransformResources={
        'InstanceType': 'ml.m5.xlarge',
        'InstanceCount': 1
    }
)

In [3]:
df_pred = pd.read_json('../data/payload.json.out')

In [4]:
cv_list = [
    'Ainda não devolvi meu produto e quero saber sobre processo de troca / devolução',
    'Estou arrependido, quero cancelar/trocar o produto',
    'Como utilizar o cupom/vale'
]

df['cv'] = df.apply(lambda x: x.sac_customer_voice if x.sac_customer_voice in (cv_list) else 'Outro', axis=1)

In [5]:
map = {
    'Ainda não devolvi meu produto e quero saber sobre processo de troca / devolução': 22,
    'Estou arrependido, quero cancelar/trocar o produto': 32,
    'Como utilizar o cupom/vale': 748,
    'Outro': 0
}

inv_map = {v: k for k, v in map.items()}

In [6]:
df_pred['cv'] = df_pred['idTemplate'].map(inv_map)

In [7]:
final_df = df.join(df_pred, lsuffix='_original', rsuffix='_predicted')
final_df

,sac_customer_voice,message,problem,detail,cv_original,isAutomatic,idTemplate,cv_predicted
0,Meu pagamento foi aprovado?,"bom dia, gostaria de cancelar uma compra feita...",Vales e reembolsos,Problemas com o estorno/reembolso,Outro,0,0,Outro
1,Ainda não coletaram meu produto,Bom dia! Aguardo resposta quanto a solicitação...,"Trocar, cancelar ou devolver",Quero trocar meu produto,Outro,0,0,Outro
2,Meu pedido ainda não chegou,Boa tarde! Meu pedido consta como entrega até ...,Dúvida sobre o andamento do meu pedido,None,Outro,0,0,Outro
3,Meu pedido ainda não chegou,Passei o dia em casa esperando a entrega que d...,Problemas com a entrega,None,Outro,0,0,Outro
4,Não consigo alterar meu cadastro,Boa noite!Gostaria de saber o que pode estar o...,Outros,None,Outro,0,0,Outro
...,...,...,...,...,...,...,...,...
44416,Não consigo verificar a Nota Fiscal/Rastreamento,"Bom dia, novamente gostaria de saber sobre o a...",Problemas com a entrega,None,Outro,0,0,Outro
44417,Já devolvi meu produto e quero saber o andamen...,Postado em 26/12. Ainda não obtive nenhum reto...,Andamento da minha troca,None,Outro,0,0,Outro
44418,"Pedido consta como entregue, mas não recebi","Bom dia, gostaria de saber quando meu pedido v...",Problemas com a entrega,None,Outro,0,0,Outro
44419,"Estou arrependido, quero cancelar/trocar o pro...","Boa tarde,Preciso fazer a devolução deste prod...","Trocar, cancelar ou devolver",Quero cancelar ou devolver minha compra,"Estou arrependido, quero cancelar/trocar o pro...",0,0,Outro


In [9]:
aux = final_df[final_df['isAutomatic'] == 1].reset_index(drop=True)

print(classification_report(final_df['cv_original'], final_df['cv_predicted']))
print(confusion_matrix(final_df['cv_original'], final_df['cv_predicted']))

                                                                                 precision    recall  f1-score   support

Ainda não devolvi meu produto e quero saber sobre processo de troca / devolução       0.62      0.26      0.37      6076
                                                     Como utilizar o cupom/vale       0.32      0.31      0.31      2679
                             Estou arrependido, quero cancelar/trocar o produto       0.34      0.35      0.34       997
                                                                          Outro       0.82      0.91      0.86     34669

                                                                       accuracy                           0.77     44421
                                                                      macro avg       0.53      0.46      0.47     44421
                                                                   weighted avg       0.75      0.77      0.75     44421

[[ 1597    46    86  4347]
 [

In [14]:
aux

,sac_customer_voice,message,problem,detail,cv_original,isAutomatic,idTemplate,cv_predicted
0,Não recebi meu reembolso/estorno/vale,Bom dia. Fiz uma compra em que o mesmo ficou e...,Outros,None,Outro,1,748,Como utilizar o cupom/vale
1,Não consigo utilizar meu vale,"Solicito o código do vale no valor de R$7,10, ...",Vales e Reembolso,Problemas com o Vale,Outro,1,748,Como utilizar o cupom/vale
2,Ja realizei a compra e estou com dúvida sobre ...,Bom dia! Ontem fiz a compra de um kit de bolsa...,Outros,None,Outro,1,32,"Estou arrependido, quero cancelar/trocar o pro..."
3,Não recebi meu reembolso/estorno/vale,Não recebi meu vale.,Vales e Reembolso,Não recebi meu vale,Outro,1,748,Como utilizar o cupom/vale
4,Como utilizar o cupom/vale,Poderia me reenviar o vale referente ao pedido...,"Trocar, cancelar ou devolver",Quero trocar meu produto,Como utilizar o cupom/vale,1,748,Como utilizar o cupom/vale
...,...,...,...,...,...,...,...,...
6214,Já devolvi meu produto mas não fui atendido,"Olá,Recentemente enviei uma mensagem para a em...",Vales e Reembolso,Não recebi meu vale,Outro,1,748,Como utilizar o cupom/vale
6215,Como utilizar o cupom/vale,quero saber se ja esta liberado meu vale troca...,Vales e Reembolso,Não recebi meu vale,Como utilizar o cupom/vale,1,748,Como utilizar o cupom/vale
6216,Como utilizar o cupom/vale,Boa tardeN consegui localiy o email com meu va...,Vales e Reembolso,Não recebi meu vale,Como utilizar o cupom/vale,1,748,Como utilizar o cupom/vale
6217,Ainda não coletaram meu produto,"Bom dia , tudo bem?eu solicitei uma troca dia ...",Andamento da minha troca,None,Outro,1,22,Ainda não devolvi meu produto e quero saber so...


In [16]:
print(aux.iloc[6214])
print(aux.iloc[6214]['message'])

sac_customer_voice          Já devolvi meu produto mas não fui atendido
message               Olá,Recentemente enviei uma mensagem para a em...
problem                                               Vales e Reembolso
detail                                              Não recebi meu vale
cv_original                                                       Outro
isAutomatic                                                           1
idTemplate                                                          748
cv_predicted                                 Como utilizar o cupom/vale
Name: 6214, dtype: object
Olá,Recentemente enviei uma mensagem para a empresa de vocês, sobre o andamento do meu pedido de troca, me informaram que foi feita uma solicitação ao setor responsável e me eram uma prazo de 3 dias. O prazo já passou e continuo na mesma situação de não receber o vale troca.Aconteceu alguma coisa com o produto postado que não tenho direito em receber o vale? Pelo sie do Correios a encomenda já e